In [5]:
import speech_recognition as sr

In [6]:
r=sr.Recognizer()

In [7]:
# reading audio file
harvard = sr.AudioFile('harvard.wav')
with harvard as source:
    audio = r.record(source)

In [8]:
#The context manager opens the file and reads its contents, storing the data in an AudioFile instance called source. 
#Then the record() method records the data from the entire file into an AudioData instance. 
#You can confirm this by checking the type of audio:
type(audio)

speech_recognition.AudioData

In [11]:
#You can now invoke recognize_google() to attempt to recognize any speech in the audio. 
#Depending on your internet connection speed, 
#you may have to wait several seconds before seeing the result.
r.recognize_google(audio)

'the salesman old beer drinkers it takes hi to bring out the order I called them restore selfinvest a salt ACL this find him because of my favourite is just for food is Bihar cross bun'

In [10]:
harvard = sr.AudioFile('OSR_us_000_0018_8k.wav')
with harvard as source:
    audio = r.record(source)

r.recognize_google(audio)    

FileNotFoundError: [Errno 2] No such file or directory: 'OSR_us_000_0018_8k.wav'

In [11]:
#The Effect of Noise on Speech Recognition
#This file has the phrase “the stale smell of old beer lingers” spoken with a loud jackhammer in the background.
jackhammer = sr.AudioFile('jackhammer.wav')
with jackhammer as source:
    audio = r.record(source)
r.recognize_google(audio)

'smell during periods'

In [ ]:
#So how do you deal with this? One thing you can try is using the adjust_for_ambient_noise() method of the Recognizer class.
with jackhammer as source:
    r.adjust_for_ambient_noise(source)
    audio = r.record(source)
r.recognize_google(audio)

In [ ]:
with jackhammer as source:
    r.adjust_for_ambient_noise(source, duration=0.4)
    audio = r.record(source)
r.recognize_google(audio,show_all=True)

In [ ]:
#Working with Microphone
#Installing PyAudio


In [ ]:
#testing the installation from console
### $ python -m speech_recognition
#Make sure your default microphone is on and unmuted. If the installation worked, you should see something like this:

#A moment of silence, please...
#Set minimum energy threshold to 600.4452854381937
#Say something!

#Go ahead and play around with it a little bit by speaking into your microphone 
#and seeing how well SpeechRecognition transcribes your speech.

In [13]:
mic = sr.Microphone()

In [14]:
#list all microphones
sr.Microphone.list_microphone_names()

# use anymicrophone as below
# mic = sr.Microphone(device_index=3)

['Microsoft Sound Mapper - Input',
 'Microphone (2- High Definition ',
 'Microsoft Sound Mapper - Output',
 ' (Intel(R) Display Audio)',
 'Speakers (2- High Definition Au']

In [15]:
#Using listen() to Capture Microphone Input
#Just like the AudioFile class, Microphone is a context manager.
#You can capture input from the microphone using the listen() method of the Recognizer class inside of the with block.
#This method takes an audio source as its first argument and records input from the source until silence is detected.

with mic as source:
    audio = r.listen(source)
    
r.recognize_google(audio)    

KeyboardInterrupt: 

In [21]:
#If the prompt never returns, your microphone is most likely picking up too much ambient noise.
with mic as source:
    r.adjust_for_ambient_noise(source)
    audio = r.listen(source)

In [22]:
r.recognize_google(audio)

"good day it's raining outside I am feeling very good"

In [28]:
with sr.Microphone() as source:
    print("Say Something dude")
    r.adjust_for_ambient_noise(source)
    audio=r.listen(source)
    print("Time over buddy")
    
r.recognize_google(audio)


Say Something dude
Time over buddy


'hello how are you what are you doing'

In [35]:
#guess a word game
import random
import time

import speech_recognition as sr


def recognize_speech_from_mic(recognizer, microphone):
    """Transcribe speech from recorded from `microphone`.

    Returns a dictionary with three keys:
    "success": a boolean indicating whether or not the API request was
               successful
    "error":   `None` if no error occured, otherwise a string containing
               an error message if the API could not be reached or
               speech was unrecognizable
    "transcription": `None` if speech could not be transcribed,
               otherwise a string containing the transcribed text
    """
    # check that recognizer and microphone arguments are appropriate type
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    # adjust the recognizer sensitivity to ambient noise and record audio
    # from the microphone
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    # set up the response object
    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    # try recognizing the speech in the recording
    # if a RequestError or UnknownValueError exception is caught,
    #     update the response object accordingly
    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        # API was unreachable or unresponsive
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        # speech was unintelligible
        response["error"] = "Unable to recognize speech"

    return response


if __name__ == "__main__":
    # set the list of words, maxnumber of guesses, and prompt limit
    WORDS = ["apple", "banana", "grape", "orange", "mango", "lemon"]
    NUM_GUESSES = 3
    PROMPT_LIMIT = 5

    # create recognizer and mic instances
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    # get a random word from the list
    word = random.choice(WORDS)

    # format the instructions string
    instructions = (
        "I'm thinking of one of these words:\n"
        "{words}\n"
        "You have {n} tries to guess which one.\n"
    ).format(words=', '.join(WORDS), n=NUM_GUESSES)

    # show instructions and wait 3 seconds before starting the game
    print(instructions)
    time.sleep(3)

    for i in range(NUM_GUESSES):
        # get the guess from the user
        # if a transcription is returned, break out of the loop and
        #     continue
        # if no transcription returned and API request failed, break
        #     loop and continue
        # if API request succeeded but no transcription was returned,
        #     re-prompt the user to say their guess again. Do this up
        #     to PROMPT_LIMIT times
        for j in range(PROMPT_LIMIT):
            print('Guess {}. Speak!'.format(i+1))
            guess = recognize_speech_from_mic(recognizer, microphone)
            if guess["transcription"]:
                break
            if not guess["success"]:
                break
            print("I didn't catch that. What did you say?\n")

        # if there was an error, stop the game
        if guess["error"]:
            print("ERROR: {}".format(guess["error"]))
            break

        # show the user the transcription
        print("You said: {}".format(guess["transcription"]))

        # determine if guess is correct and if any attempts remain
        guess_is_correct = guess["transcription"].lower() == word.lower()
        user_has_more_attempts = i < NUM_GUESSES - 1

        # determine if the user has won the game
        # if not, repeat the loop if user has more attempts
        # if no attempts left, the user loses the game
        if guess_is_correct:
            print("Correct! You win!".format(word))
            break
        elif user_has_more_attempts:
            print("Incorrect. Try again.\n")
        else:
            print("Sorry, you lose!\nI was thinking of '{}'.".format(word))
            break

I'm thinking of one of these words:
apple, banana, grape, orange, mango, lemon
You have 3 tries to guess which one.

Guess 1. Speak!
You said: banana
Incorrect. Try again.

Guess 2. Speak!
You said: orange
Incorrect. Try again.

Guess 3. Speak!
You said: Apple
Sorry, you lose!
I was thinking of 'grape'.
